## Introduction to Linear Regression

We'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [5]:
import numpy as np
import torch

## Training Data

We can represents the training data using two matrics : `inputs` and `targets`, each with one row per observation, and one column per variable.

In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73,67,43],
                    [91,88,64],
                    [87,134,58],
                    [102,43,37],
                    [69,96,70]], dtype = 'float32')

In [7]:
# Targets (apples, oranges)
targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [103,119]], dtype = 'float32')

Let's convert array to Pytorch tensor.

In [8]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
inputs, targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [14]:
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
w, b

(tensor([[ 0.4646,  2.3230, -0.2447],
         [-0.3319, -1.1212, -0.5929]], requires_grad=True),
 tensor([-2.0672, -0.4643], requires_grad=True))

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [15]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in Pytorch and `.t()` represents transpose of a tensor.

In [16]:
# Generates predictions
preds = model(inputs)
preds

tensor([[ 176.9684, -125.3101],
        [ 228.9754, -167.2807],
        [ 335.4420, -213.9706],
        [ 136.1600, -104.4705],
        [ 235.8686, -172.5048]], grad_fn=<AddBackward0>)

In [17]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [18]:
# MSE Loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [19]:
# Compute loss
loss = mse(preds, targets)
loss

tensor(44956.1836, grad_fn=<DivBackward0>)

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [20]:
# Compute gradients
loss.backward()

In [21]:
# Gradients for weights
print(w)
print(w.grad) #dl/dw

tensor([[ 0.4646,  2.3230, -0.2447],
        [-0.3319, -1.1212, -0.5929]], requires_grad=True)
tensor([[ 12387.8330,  13558.8418,   8150.0840],
        [-20680.2871, -23451.2441, -14266.4678]])


In [22]:
# Gradients for bias
print(b)
print(b.grad) #dl/db

tensor([-2.0672, -0.4643], requires_grad=True)
tensor([ 146.4829, -248.7073])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [23]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [27]:
# Let's verify that the loss is actually lower
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(30525.0664, grad_fn=<DivBackward0>)

Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [28]:
w.grad.zero_()
b.grad.zero_()
w.grad, b.grad

(tensor([[0., 0., 0.],
         [0., 0., 0.]]),
 tensor([0., 0.]))

## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [29]:
# Generate Predictions
preds = model(inputs)
preds

tensor([[ 155.3348,  -88.3641],
        [ 200.5531, -118.6915],
        [ 301.7672, -156.2770],
        [ 114.6771,  -68.0115],
        [ 208.5980, -125.7332]], grad_fn=<AddBackward0>)

In [30]:
# Calculate the loss
loss = mse(preds, targets)
loss

tensor(30525.0664, grad_fn=<DivBackward0>)

In [31]:
# Compute the gradients
loss.backward()
w.grad, b.grad

(tensor([[ 10154.1709,  11157.8877,   6668.8418],
         [-16863.4746, -19343.2500, -11732.9463]]),
 tensor([ 119.9861, -203.4155]))

In [33]:
# Adjust the weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [34]:
# Let's take a look at the new weights & biases
w,  b

(tensor([[ 0.2392,  2.0758, -0.3929],
         [ 0.0435, -0.6932, -0.3329]], requires_grad=True),
 tensor([-2.0699, -0.4598], requires_grad=True))

With the new weights and biases, the model should have a lower loss.

In [36]:
# Calculate the loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(20797.7207, grad_fn=<DivBackward0>)

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [57]:
# Train for 100 epochs
for i in range(2500):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [58]:
# Let's verify that the loss is now lower
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(0.4965, grad_fn=<DivBackward0>)

In [59]:
preds, targets

(tensor([[ 56.8190,  70.2313],
         [ 82.4240, 100.7189],
         [118.7495, 132.9712],
         [ 21.1252,  37.0249],
         [101.8648, 119.1254]], grad_fn=<AddBackward0>),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [60]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [61]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [64]:
from torch.utils.data import TensorDataset

# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [67]:
from torch.utils.data import DataLoader

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [72]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 88., 134.,  59.],
        [ 68.,  96.,  71.],
        [ 91.,  88.,  64.],
        [ 68.,  97.,  70.],
        [ 91.,  87.,  65.]])
tensor([[118., 132.],
        [104., 118.],
        [ 81., 101.],
        [102., 120.],
        [ 80., 102.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [73]:
model = nn.Linear(3, 2)
model.weight, model.bias

(Parameter containing:
 tensor([[-0.2313,  0.5382,  0.2293],
         [ 0.0175, -0.2791,  0.4466]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2401, -0.3807], requires_grad=True))

PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [74]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2313,  0.5382,  0.2293],
         [ 0.0175, -0.2791,  0.4466]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2401, -0.3807], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [75]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 29.2756,   1.4034],
        [ 41.2303,   5.2365],
        [ 65.5389, -10.3522],
        [  8.2734,   5.9309],
        [ 52.0014,   5.2975],
        [ 28.5060,   1.7000],
        [ 40.9214,   5.9622],
        [ 65.5368,  -9.8881],
        [  9.0430,   5.6343],
        [ 52.4620,   5.7265],
        [ 28.9667,   2.1291],
        [ 40.4607,   5.5332],
        [ 65.8478, -11.0779],
        [  7.8127,   5.5018],
        [ 52.7709,   5.0008]], grad_fn=<AddmmBackward>)

## Loss Function

In [76]:
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilites.

In [78]:
loss_fn = F.mse_loss

# Let's compute the loss for current predictions
loss = loss_fn(model(inputs), targets)
loss

tensor(5607.2812, grad_fn=<MseLossBackward>)

## Optimizer

In [79]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

In [99]:
def fit(num, model, loss_fn, opt,train_dl):
    for epoch in range(num):
        for xb, yb in train_dl: # Train with batches of data
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
            if (epoch+1) % 100 == 0:
                print('Epoch [{}/{}], Loss : {:.4f}'.format(epoch+1, num, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model.

In [88]:
fit(1000, model, loss_fn, opt, train_dl)

Epoch [100/1000], Loss : 0.9327
Epoch [100/1000], Loss : 1.0374
Epoch [100/1000], Loss : 1.0921
Epoch [200/1000], Loss : 1.4317
Epoch [200/1000], Loss : 0.7702
Epoch [200/1000], Loss : 0.8674
Epoch [300/1000], Loss : 1.2599
Epoch [300/1000], Loss : 0.8106
Epoch [300/1000], Loss : 1.0335
Epoch [400/1000], Loss : 0.7498
Epoch [400/1000], Loss : 0.9025
Epoch [400/1000], Loss : 1.4357
Epoch [500/1000], Loss : 1.2169
Epoch [500/1000], Loss : 0.8513
Epoch [500/1000], Loss : 0.9867
Epoch [600/1000], Loss : 1.2768
Epoch [600/1000], Loss : 1.0747
Epoch [600/1000], Loss : 0.7614
Epoch [700/1000], Loss : 0.7659
Epoch [700/1000], Loss : 1.0723
Epoch [700/1000], Loss : 1.2750
Epoch [800/1000], Loss : 1.4725
Epoch [800/1000], Loss : 0.9056
Epoch [800/1000], Loss : 0.7389
Epoch [900/1000], Loss : 1.1928
Epoch [900/1000], Loss : 1.0878
Epoch [900/1000], Loss : 0.7974
Epoch [1000/1000], Loss : 0.9574
Epoch [1000/1000], Loss : 1.2053
Epoch [1000/1000], Loss : 0.9411


Let's generate predictions using our model and verify that they're close to our targets.

In [91]:
preds = model(inputs)
print(preds)
targets

tensor([[ 56.9783,  70.3440],
        [ 81.9945, 100.5987],
        [118.1534, 132.8231],
        [ 20.8007,  37.7761],
        [101.8587, 118.5105],
        [ 55.7301,  69.2688],
        [ 81.8507, 100.6977],
        [118.4448, 133.4312],
        [ 22.0490,  38.8513],
        [102.9631, 119.6848],
        [ 56.8345,  70.4430],
        [ 80.7463,  99.5234],
        [118.2972, 132.7241],
        [ 19.6963,  36.6019],
        [103.1070, 119.5858]], grad_fn=<AddmmBackward>)


tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Let's try single input

In [92]:
model(torch.tensor([75,63,44.]))

tensor([53.4963, 67.5004], grad_fn=<AddBackward0>)

The predicted yield of apples is 53.49 tons per hectare, and that of oranges is 67.5 tons per hectare.

In [101]:
model2 = nn.Sequential(
    nn.Linear(3, 4),
    nn.Sigmoid(),
    nn.Linear(4,2)
)

In [102]:
opt2 = torch.optim.SGD(model2.parameters(), lr = 1e-3)

In [103]:
fit(1000, model2, F.mse_loss, opt2, train_dl)

Epoch [100/1000], Loss : 1571.5996
Epoch [100/1000], Loss : 1571.2960
Epoch [100/1000], Loss : 1537.9662
Epoch [200/1000], Loss : 1073.0765
Epoch [200/1000], Loss : 1428.6136
Epoch [200/1000], Loss : 1133.9960
Epoch [300/1000], Loss : 1193.1929
Epoch [300/1000], Loss : 1038.9900
Epoch [300/1000], Loss : 1344.4264
Epoch [400/1000], Loss : 1262.5671
Epoch [400/1000], Loss : 1193.8103
Epoch [400/1000], Loss : 1116.0923
Epoch [500/1000], Loss : 1042.5502
Epoch [500/1000], Loss : 1069.0898
Epoch [500/1000], Loss : 1460.7181
Epoch [600/1000], Loss : 531.7107
Epoch [600/1000], Loss : 1282.6626
Epoch [600/1000], Loss : 1760.1846
Epoch [700/1000], Loss : 1534.9963
Epoch [700/1000], Loss : 1311.2501
Epoch [700/1000], Loss : 730.0565
Epoch [800/1000], Loss : 1701.8879
Epoch [800/1000], Loss : 671.1722
Epoch [800/1000], Loss : 1203.8004
Epoch [900/1000], Loss : 1249.0642
Epoch [900/1000], Loss : 1132.1892
Epoch [900/1000], Loss : 1191.7000
Epoch [1000/1000], Loss : 856.5430
Epoch [1000/1000], Loss

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

